In [27]:
from src.models import BERTGlobalPointer
import torch
import numpy as np
import spacy
from spacy.lang.zh import Chinese
import srsly
from tqdm import tqdm

In [41]:
model = BERTGlobalPointer.load_from_checkpoint('/home/vhels/project/deep-lab/logs/experiments/CMEee_charspan/runs/2022-01-23/10-11-26/checkpoints/epoch08_0.660.ckpt')

In [3]:
# model.freeze()

In [4]:
# spans = model.predict('（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。')
# spans

In [5]:
nlp = spacy.blank('zh')
sentcut = nlp.add_pipe('sentencizer')
sentcut.punct_chars.add(';')
sentcut.punct_chars.add('；')

In [6]:
class SentSpancat:
    def __init__(self, ckpt):
        self.model = BERTGlobalPointer.load_from_checkpoint(ckpt)
        self.model.freeze()
        
    def __call__(self, doc):
        all_spans = []
        for sent in doc.sents:
            spans = self.model.predict(sent.text)
            for span in spans:
                s = sent.char_span(span[0], span[1]+1, span[2])
                all_spans.append(s)
        doc.spans['all'] = all_spans
        return doc
@Chinese.factory('sent_spancat')
def make_spancat(nlp, name):
    return SentSpancat(ckpt='/home/vhels/project/deep-lab/logs/experiments/CMEee_charspan/runs/2022-01-23/10-11-26/checkpoints/epoch08_0.660.ckpt')

In [7]:
nlp.add_pipe('sent_spancat')

In [9]:
doc = nlp('（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。')

In [10]:
doc.spans

{'all': [反复发作或难治性心房内折返性心动过速, 房室结消融, 起搏器植入]}

In [28]:
test_data = srsly.read_json('data/CMeEE_test.json')

In [36]:
new_data = []
for d in tqdm(test_data):
    d['entities'] = []
    doc = nlp(d['text'])
    spans = doc.spans['all']
    for span in spans:
        d['entities'].append({'start_idx':span.start_char, 'end_idx':span.end_char-1, 'type':span.label_, 'entity':span.text})
    new_data.append(d)

100%|██████████| 3000/3000 [05:15<00:00,  9.51it/s]


In [38]:
srsly.write_json('CMeEE_test.json', new_data)

In [39]:
!zip CMeEE_test.zip CMeEE_test.json

  adding: CMeEE_test.json (deflated 86%)


In [37]:
new_data[0]

{'text': '六、新生儿疾病筛查的发展趋势自1961年开展苯丙酮尿症筛查以来，随着医学技术的发展，符合进行新生儿疾病筛查标准的疾病也在不断增加，无论在新生儿疾病筛查的病种，还是在新生儿疾病筛查的技术方法上，都有了非常显著的进步。',
 'entities': [{'start_idx': 2,
   'end_idx': 8,
   'type': 'pro',
   'entity': '新生儿疾病筛查'},
  {'start_idx': 22, 'end_idx': 28, 'type': 'pro', 'entity': '苯丙酮尿症筛查'},
  {'start_idx': 46, 'end_idx': 52, 'type': 'pro', 'entity': '新生儿疾病筛查'},
  {'start_idx': 68, 'end_idx': 74, 'type': 'pro', 'entity': '新生儿疾病筛查'},
  {'start_idx': 82, 'end_idx': 88, 'type': 'pro', 'entity': '新生儿疾病筛查'}]}

In [44]:
model.hparams

"batch_size":       12
"data_dir":         /home/vhels/project/deep-lab/data/
"dataset":          cblue_ee_charspan
"dropout":          0.2
"hidden_size":      256
"label2id":         {'bod': 0, 'dep': 1, 'dis': 2, 'dru': 3, 'equ': 4, 'ite': 5, 'mic': 6, 'pro': 7, 'sym': 8}
"lr":               3e-05
"max_length":       420
"num_workers":      10
"pin_memory":       True
"pretrained_dir":   /home/vhels/project/deep-lab/pretrained_models/
"pretrained_model": chinese-roberta-wwm-ext
"storer":           oss
"weight_decay":     0.0